In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from datetime import datetime 
from google.oauth2 import service_account

import functions_framework

from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest
import os


In [9]:
# @functions_framework.http
# def load_new_incident_ml(request):

In [5]:
projectId='smart-data-ml'
start_date_query=os.environ.get('start_date_query', '2023-03-13')
start_date_query='2023-03-13'

In [6]:
projectId='smart-data-ml'

table_ml_id = f"{projectId}.SMartML.new_incident"
table_dw_id=f"{projectId}.SMartDW.incident"

# credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
# client = bigquery.Client(credentials=credentials, project=projectId)

client = bigquery.Client(project=project=projectId)

In [88]:
# Get Last Upldate from BQ update data
dateCols=['open_datetime','close_datetime','response_datetime','resolved_datetime']

removeCols=dateCols+['open_to_close','response_to_resolved']

numbericCols=['open_to_close_hour','response_to_resolved_hour']
cateCols=['sla','product_type','brand','service_type','incident_type']

file_name="New_Incident.csv"


In [89]:
#https://cloud.google.com/bigquery/docs/samples/bigquery-create-table#bigquery_create_table-python

try:
    client.get_table(table_ml_id)  # Make an API request.
    print("Table {} already exists.".format(table_ml_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("sla", "STRING", mode="REQUIRED"),    
    bigquery.SchemaField("product_type", "STRING", mode="REQUIRED"),  
    bigquery.SchemaField("brand", "STRING", mode="REQUIRED"),  
    bigquery.SchemaField("service_type", "STRING", mode="REQUIRED"),  
    bigquery.SchemaField("incident_type", "STRING", mode="REQUIRED"),  
    bigquery.SchemaField("open_to_close_hour", "FLOAT", mode="REQUIRED"),
    bigquery.SchemaField("response_to_resolved_hour", "FLOAT", mode="REQUIRED"),    
    bigquery.SchemaField("imported_at", "DATETIME", mode="REQUIRED")    
    ]

    table = bigquery.Table(table_ml_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="imported_at")

    
    table = client.create_table(table)  # Make an API request.
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )


Table smart-data-ml.SMartML.new_incident already exists.


Imported DateTime: 2023-03-31 22:36:07


In [74]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )

sql_lastImport=f"SELECT max(imported_at) as last_imported from `{table_ml_id}` "
job_lastImported=client.query(sql_lastImport)
str_lastImported=None
for row in job_lastImported:    
    if row.last_imported is not None: 
        str_lastImported=row.last_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Last Imported DateTime: {str_lastImported}" )

if str_lastImported is not None:
  print("Start date from last loading")  
  start_date_query=str_lastImported
else:
  print("Init First loading")  


print(f"Start Import on update_at of last imported date : {start_date_query}" )


Imported DateTime: 2023-03-31 22:39:35
Last Imported DateTime: 2023-03-31 01:00:04
Start date from last loading
Start Import on update_at of last imported date : 2023-03-31 01:00:04


In [75]:
sql=f"""
SELECT  id,
severity_id,severity_name,sla,
product_type,brand,service_type,incident_type,
open_datetime,  close_datetime, response_datetime,resolved_datetime
FROM `{table_dw_id}` 
WHERE imported_at>'{start_date_query}'
order by imported_at
"""
#WHERE imported_at>='{start_date_query}' and imported_at<='2023-03-24'
#WHERE imported_at>='{start_date_query}'

query_result=client.query(sql)
df_all=query_result.to_dataframe()
print(df_all.info())
df_all.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 2 non-null      Int64         
 1   severity_id        2 non-null      Int64         
 2   severity_name      2 non-null      object        
 3   sla                2 non-null      object        
 4   product_type       2 non-null      object        
 5   brand              2 non-null      object        
 6   service_type       2 non-null      object        
 7   incident_type      2 non-null      object        
 8   open_datetime      2 non-null      datetime64[ns]
 9   close_datetime     2 non-null      datetime64[ns]
 10  response_datetime  2 non-null      datetime64[ns]
 11  resolved_datetime  2 non-null      datetime64[ns]
dtypes: Int64(2), datetime64[ns](4), object(6)
memory usage: 324.0+ bytes
None


,id,severity_id,severity_name,sla,product_type,brand,service_type,incident_type,open_datetime,close_datetime,response_datetime,resolved_datetime
0,2524,2,Major,8x5 4Hrs Response Time,Software,Microsoft,Request,General Incident,2023-03-26 09:00:00,2023-03-27 01:00:00,2023-03-26 09:30:00,2023-03-26 23:00:00
1,2575,2,Major,24x7 6Hrs Resolution Time,Software,YIP,Request,Software,2023-03-31 12:00:00,2023-03-31 16:53:00,2023-03-31 13:10:00,2023-03-31 16:53:00


In [76]:
if len(df_all)==0:
 print("No record to load")   
 return "No record to load"  

In [77]:
start_end_list=[ ['open_datetime','close_datetime'],['response_datetime','resolved_datetime']]
listDiffDateDeltaCols=[]
listDiffHourCols=[]
for item  in  start_end_list:
   diff_str=f"{item[0]}_to_{item[1]}" 
   diff_str=diff_str.replace('_datetime','')  
   listDiffDateDeltaCols.append(diff_str)
   df_all[diff_str]=df_all[item[1]]-df_all[item[0]]
    
   diff_hour=f'{diff_str}_hour'
   listDiffHourCols.append(diff_hour)
   df_all[diff_hour] = df_all[diff_str].apply(lambda x:  x.total_seconds() / (60*60) if x is not np.nan else np.nan  )


In [82]:
for col in numbericCols:
 df_all=df_all.query(f'{col}!=0')

df_all=df_all.drop_duplicates(subset=['id'],keep='first')
df_all=df_all.drop(columns=removeCols)

df_all['imported_at']=dt_imported

df_all.dropna(inplace=True)

print(df_all.info())
print(df_all.tail())

KeyError: "['open_datetime', 'close_datetime', 'response_datetime', 'resolved_datetime', 'open_to_close', 'response_to_resolved'] not found in axis"

In [79]:
# print(df_all[numbericCols].describe(percentiles=[.9,.75,.50,.25,.10]))

In [80]:
# df_all.to_csv("data/New_Incident.csv",index=False)


In [81]:

def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            df_all, table_ml_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total ", len(df_all), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex


Total  2 Imported igquery successfully


In [ ]:
# return 'ok'